# AWS Academy Machine Learning Foundations - Laboratório do Amazon Forecast

## PARE! - Este arquivo é executado automaticamente na inicialização do laboratório. Não tente executar as células!

Este notebook Jupyter faz parte do laboratório de alunos do Amazon Forecast. Ele foi executado quando o laboratório foi criado.

Se você acabou de iniciar o laboratório, carregue `forecast-lab.ipynb` e trabalhe nesse notebook, em vez de utilizar este.

Se você foi direcionado para esse arquivo, leia as células e as explicações. Evite executar as células.


## Resumo do notebook

Este notebook carrega e pré-processa o conjunto de dados (dataset) de varejo on-line. Os dados são carregados no Amazon Simple Storage Service (Amazon S3), onde são usados para criar uma previsão por meio do Amazon Forecast. O notebook executa as seguintes etapas:

- **Importação e funções** importa os pacotes usados e cria funções auxiliares.
- **Importação de dados** faz download e carrega os dados em um DataFrame pandas.
- **Pré-processamento de dados** filtra os dados que estão prontos para treinamento
- **Geração de DataFrames de treinamento e de teste** faz um downsamples dos dados para uma frequência diária e divide o conjunto de dados (dataset) em DataFrames de treinamento e teste.
- **Upload para o Amazon S3** faz upload dos DataFrames para o Amazon S3 como arquivos CSV (valores separados por vírgula).
- **Criação do grupo de conjuntos de dados (dataset) do Amazon Forecast** cria o grupo de conjuntos de dados (dataset) do projeto.
- **Criação dos conjuntos de dados (dataset)** cria os conjuntos de dados (datasets) no grupo de conjuntos de dados (dataset groups) e aguarda a conclusão da importação.
- **Criação do previsor** treina o previsor usando o grupo de conjuntos de dados.
- **Obtenção de métricas de precisão** exibe as métricas para o previsor.
- **Criação do previsor** treina o previsor usando o grupo de conjuntos de dados (datasets).
- **Limpeza opcional** pode executar uma limpeza se ela não for concluída no notebook `forecast-lab.ipynb`.

Este notebook leva entre 60 e 90 minutos para ser executado.


## Importação e funções

O código a seguir importa esses pacotes:

- *boto3* representa o AWS SDK para Python (Boto3), que é a biblioteca Python da AWS
- *pandas* fornece DataFrames para manipular dados de séries temporais
- *matplotlib* fornece funções de plotagem
- *sagemaker* representa a API necessária para trabalhar com o Amazon SageMaker
- *time*,*sys*,*os*,*io* e *json* fornecem funções auxiliares 

Além disso, duas funções auxiliares são criadas:

- `upload_s3_csv` faz upload de DataFrames pandas para o Amazon S3 como arquivos CSV. O cabeçalho é removido, mas *não* o índice.
- `StatusIndicator` fornece uma função de status para chamadas de API de longa execução para o Amazon Forecast.



In [ ]:
import warnings
warnings.filterwarnings('ignore')
bucket_name='c33334a421003l780155t1w26178775763-forecastbucket-y617mqtzdoyg'

import boto3
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time, sys, os, io, json
import sagemaker

%store bucket_name

s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(filename, header=False, index=True)
    dataframe.to_csv(csv_buffer, header=False, index=True )
    s3_resource.Bucket(bucket_name).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

class StatusIndicator:
    
    def __init__(self):
        self.previous_status = None
        self.need_newline = False
        
    def update( self, status ):
        if self.previous_status != status:
            if self.need_newline:
                sys.stdout.write("\n")
            sys.stdout.write( status + " ")
            self.need_newline = True
            self.previous_status = status
        else:
            sys.stdout.write(".")
            self.need_newline = True
        sys.stdout.flush()

    def end(self):
        if self.need_newline:
            sys.stdout.write("\n")

## Importação de dados

A célula a seguir faz download do conjunto de dados, que é um arquivo do Microsoft Excel. Esse arquivo é carregado no pandas como um DataFrame.

In [ ]:

session = boto3.Session()
forecast = session.client(service_name='forecast') 
forecast_query = session.client(service_name='forecastquery')

!aws s3 cp s3://aws-tc-largeobjects/CUR-TF-200-ACMLFO-1/forecast/ . --recursive


## Pré-processamento de dados

A célula a seguir conclui as seguintes etapas de pré-processamento:

- Remove instâncias com valores ausentes
- Define o índice para o recurso InvoiceDate
- Remove instâncias que não são do Reino Unido
- Remove instâncias que não usam o código de estoque de destino (21232)
- Mantém instâncias em que o preço é maior que 0



In [ ]:
retail = pd.read_excel('online_retail_II.xlsx')
retail = retail.dropna()
retail['InvoiceDate'] = pd.to_datetime(retail.InvoiceDate)
retail = retail.set_index('InvoiceDate')

country_filter = ['United Kingdom']
retail = retail[retail['Country'].isin(country_filter)]

#stockcodes = ['ADJUST', 'ADJUST2', 'POST', 'M']
#stockcodes = [21232,22423]
stockcodes = [21232]
retail = retail[retail.StockCode.isin(stockcodes)]

retail = retail[retail['Price']>0]

## Geração de DataFrames de treinamento e teste

A célula a seguir:

- Divide os dados em séries temporais e pandas DataFrames de séries temporais relacionadas.
- Faz o downsamples dos dados de vários registros de vendas por dia em um único valor diário. A coluna **Quantity** (Quantidade) é somada e a média é usada para a coluna **Price** (Preço).
- Divide os DataFrames em um conjunto de dados (datasets) de treinamento de janeiro de 2010 a outubro de 2010 e um conjunto de dados (dataset) de teste de novembro de 2010 a dezembro de 2010.



In [ ]:

retail_timeseries = retail[['StockCode','Quantity']]

retail_timeseries = retail_timeseries.groupby('StockCode').resample('D').sum().reset_index().set_index(['InvoiceDate'])

df_related_time_series = retail[['StockCode','Price']]
df_related_time_series2 = df_related_time_series.groupby('StockCode').resample('D').mean().reset_index().set_index(['InvoiceDate'])
df_related_time_series3 = df_related_time_series2.groupby('StockCode').pad()

#df_related_time_series4 = df_related_time_series3.reset_index().set_index('InvoiceDate')

# Select January to November for one DataFrame.
jan_to_oct = retail_timeseries['2009-12':'2010-10']
nov_to_dec = retail_timeseries['2010-11':'2010-12']
jan_to_oct_related = df_related_time_series3['2009-12':'2010-10']

## Upload para o Amazon S3

A célula a seguir faz upload dos DataFrames para o Amazon S3 usando a função auxiliar criada anteriormente.

In [ ]:

prefix='lab_4'
train='retail_ts_train.csv'
train_related='related_ts_train.csv'
test='retail_ts_test.csv'

key=prefix + '/forecast/' + train
# key='lab_4_forecast_t/forecast/retail_time_series_train.csv'
related_key = prefix + '/forecast/' + train_related
# related_key='lab_4_forecast_t/forecast/related.csv'

upload_s3_csv(train, 'forecast', jan_to_oct)
upload_s3_csv(train_related, 'forecast', jan_to_oct_related)
upload_s3_csv(test, 'forecast', nov_to_dec)

dataset_frequency = "D" 
timestamp_format = "yyyy-MM-dd"

# project = prefix
dataset_name = prefix+'_ds'
related_dataset_name = prefix+'_rds'
dataset_group_name = prefix +'_dsg'

s3_data_path = "s3://"+bucket_name+"/"+key
s3_related_data_path = "s3://"+bucket_name+"/"+related_key

In [ ]:

%store prefix
%store train
%store test
%store key

## Criação do grupo de conjuntos de dados (datsets group) do Amazon Forecast

A célula a seguir cria o grupo de conjuntos de dados (datasets group) para a previsão.

In [ ]:
create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=dataset_group_name,
                                                              Domain="RETAIL"
                                                             )
dataset_group_arn = create_dataset_group_response['DatasetGroupArn']

## Criação de conjuntos de dados (datasets)

A célula a seguir cria as séries temporais e os conjuntos de dados (datasets) relacionados e os adiciona ao grupo de conjuntos de dados (datasets group).

A célula aguardará o loop e exibirá o status até que os conjuntos de dados (datasets) sejam criados.

In [ ]:

#role_arn = 'arn:aws:iam::610073867519:role/service-role/AmazonForecast-ExecutionRole-1587767069972'
iam = boto3.resource('iam')
role_arn = iam.Role('forecast-role').arn

# This is the schema of the time series dataset.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"demand",
         "AttributeType":"float"
      }
   ]
}

time_series_response=forecast.create_dataset(
                    Domain="RETAIL",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=dataset_name,
                    DataFrequency=dataset_frequency, 
                    Schema = schema
)

dataset_arn = time_series_response['DatasetArn']
# forecast.describe_dataset(DatasetArn=dataset_arn)

# Create the import job for the time series dataset.
dataset_import_job_name = 'EP_DSIMPORT_JOB_TARGET'
data_source = {"S3Config" : {"Path":s3_data_path,"RoleArn": role_arn} }
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=dataset_import_job_name,
                                                          DatasetArn=dataset_arn,
                                                          DataSource= data_source,
                                                          TimestampFormat=timestamp_format
                                                         )

ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']

# This is the schema of the related data, which contains the price.
related_schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"price",
         "AttributeType":"float"
      }
   ]
}

related_time_series_response=forecast.create_dataset(
                    Domain="RETAIL",
                    DatasetType='RELATED_TIME_SERIES',
                    DatasetName=related_dataset_name,
                    DataFrequency=dataset_frequency, 
                    Schema = related_schema
)
related_dataset_arn = related_time_series_response['DatasetArn']

# forecast.describe_dataset(DatasetArn=related_dataset_arn)


related_dataset_import_job_name = 'EP_DSIMPORT_JOB_TARGET_RELATED'

related_data_source = {"S3Config" : {"Path":s3_related_data_path,"RoleArn": role_arn} }

ds_related_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=related_dataset_import_job_name,
                                                          DatasetArn=related_dataset_arn,
                                                          DataSource= related_data_source,
                                                          TimestampFormat=timestamp_format
                                                         )

ds_related_import_job_arn=ds_related_import_job_response['DatasetImportJobArn']

# Add the time series and related dataset to the dataset group.
forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=[dataset_arn, related_dataset_arn])
#forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=[dataset_arn])

# Wait for the related dataset to finish.
status_indicator = StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_related_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

# Wait for the time series dataset to finish. This process typically takes longer than the related set.
status_indicator = StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

A célula a seguir armazena os Nomes de recurso da Amazon - Amazon Resource Names (ARNs) para os objetos de previsão (forecast objects) criados anteriormente. Eles podem ser carregados de outros notebooks.

In [ ]:
%store ds_import_job_arn
%store dataset_arn
%store dataset_group_arn
%store related_dataset_arn
%store ds_related_import_job_arn

## Criação do preditor

A célula a seguir cria o preditor usando os seguintes parâmetros:

- O horizonte de previsão é definido como *30 dias*.
- *DeepAR+* é o algoritmo selecionado. Para obter mais informações, consulte [DeepAR+Algorithm] (https://docs.aws.amazon.com/forecast/latest/dg/aws-forecast-recipe-deeparplus.html) na documentação da AWS.
- Os hiperparâmetros são especificados para o algoritmo. Esses hiperparâmetros foram gerados executando a previsão com PerformHPO definido como *true*. Isso criou um trabalho de ajuste (tuning job) de hiperparâmetros no modelo, que produziu os valores a seguir.
- Uma única janela de retrocesso é usada por *30 dias*.
- A configuração de dados de entrada é definida como o grupo de conjuntos de dados (dataset) criado anteriormente.
- Feriados para o Reino Unido são adicionados como componentes (features) complementares.
- Um pipeline de caracterização de componentes é criado para os componentes (features) relativos a preços. Para obter mais informações, consulte o tópico [Tratamento de valores ausentes] (https://docs.aws.amazon.com/forecast/latest/dg/howitworks-missing-values.html) na documentação da AWS.

A célula aguardará o loop e exibirá o status até que os conjuntos de dados (datasets) sejam criados.

In [ ]:

predictor_name= prefix+'_deeparp_algo'
forecast_horizon = 30
algorithm_arn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

training_parameters =  {'context_length': '172', 
                        'epochs': '500', 
                        'learning_rate': '0.00023391131837525837', 
                        'learning_rate_decay': '0.5', 
                        'likelihood': 'student-t', 
                        'max_learning_rate_decays': '0', 
                        'num_averaged_models': '1', 
                        'num_cells': '40', 
                        'num_layers': '2', 
                        'prediction_length': '30'}

evaluation_parameters= {"NumberOfBacktestWindows": 1, "BackTestWindowOffset": 30}

input_data_config = {"DatasetGroupArn": dataset_group_arn, "SupplementaryFeatures": [ {"Name": "holiday","Value": "UK"} ]}
                  
featurization_config= {"ForecastFrequency": dataset_frequency,
                      "Featurizations": 
                      [
                          {
                            "AttributeName": "price",
                            "FeaturizationPipeline": [
                                {
                                    "FeaturizationMethodName": "filling",
                                    "FeaturizationMethodParameters": {
                                        "middlefill": "median",
                                        "backfill": "min",
                                        "futurefill": "max"               
                                        }
                                }
                            ]
                        }
                      ]}


create_predictor_response=forecast.create_predictor(PredictorName = predictor_name, 
                                                  AlgorithmArn = algorithm_arn,
                                                  ForecastHorizon = forecast_horizon,
                                                  PerformAutoML = False,
                                                  PerformHPO = False,
                                                  EvaluationParameters= evaluation_parameters, 
                                                  InputDataConfig = input_data_config,
                                                  FeaturizationConfig = featurization_config,
                                                  TrainingParameters = training_parameters
                                                 )

predictor_arn = create_predictor_response['PredictorArn']
status_indicator = StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
f = forecast.describe_predictor(PredictorArn=predictor_arn)
print(f['TrainingParameters'])

## Obtenção de métricas de precisão

A próxima célula imprime as métricas de precisão para o preditor que acabou de ser criado.

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=predictor_arn)

## Criação da previsão (forecast)

A célula a seguir cria uma previsão do preditor que foi criado anteriormente. 

Os valores de ARN do preditor e da predição são armazenados para que possam ser recuperados do notebook do laboratório.



In [ ]:
forecast_Name= prefix+'_deeparp_algo_forecast'
create_forecast_response=forecast.create_forecast(ForecastName=forecast_Name,
                                                  PredictorArn=predictor_arn)
forecast_arn = create_forecast_response['ForecastArn']

In [ ]:
%store forecast_arn
%store predictor_arn

In [ ]:
status_indicator = StatusIndicator()
while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

print(forecast_arn)

A próxima célula cria uma previsão rápida como um teste, o que pode ser útil para a solução de problemas.

In [ ]:
print()
forecast_response = forecast_query.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"21232"}
)
print(forecast_response)

## Limpeza opcional

A limpeza é realizada no notebook `forecast-lab.ipynb`. Se você precisar executar a limpeza aqui, altere a célula a seguir para código selecionando a célula e pressionando S. Em seguida, execute a célula.

forecast.delete_forecast(ForecastArn=forecast_arn)
time.sleep(60)

forecast.delete_predictor(PredictorArn=predictor_arn)
time.sleep(60)

forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)
time.sleep(60)

forecast.delete_dataset(DatasetArn=dataset_arn)
time.sleep(60)

forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn)